In [1]:
import sys
import delivery_driver as dd
sys.path.append("../../deliveryrobot")
from utilities.utilities import *
import time

In [2]:
start_time = time.time()
robot = dd.DeliveryRobot()
print(f"Robot setup complete in {time.time()-start_time} seconds")

Robot setup complete in 0.9316167831420898 seconds


In [3]:
speed = np.array([0.2,0.2])
speed *= 0.2/0.27
print(speed)
robot.robot.set_motors(speed[0],speed[1])
time.sleep(3)
robot.robot.stop()

[0.14814815 0.14814815]


In [4]:
robot.target_ai.slam_update([0.5,0.1,0])
robot.arrive_ai(1)


- - - - - - - - - - -

Time since last call: 3.0959084033966064
Steering: [0. 0.] 0.0
Position: [0. 0.]
Orientation: 0.0
Velocity: [0. 0.]
Rotation Vel: 0.0


1

In [5]:
import kinematics.movementai as ai
start_time = time.time()
steering = ai.SteeringOutput([0.2,0.0],0.0)
left, right = robot.robot_ai.get_drive_params(steering, 1)
print(left,right)
robot.robot.set_motors(left,right)
print("Duration:", time.time() - start_time)
time.sleep(0.5)
robot.robot.stop()

3.219294437326594 3.236550871655343
Duration: 0.021851062774658203


In [6]:
import matplotlib.pyplot as plt
import cv2
i=0

In [7]:
"""filename = f"calibration_{i}.jpg"
i += 1
filename = robot.take_picture(cal_dir, filename)
img = cv2.imread(filename)
plt.imshow(img)"""

'filename = f"calibration_{i}.jpg"\ni += 1\nfilename = robot.take_picture(cal_dir, filename)\nimg = cv2.imread(filename)\nplt.imshow(img)'

In [8]:
from sensors.camera.apriltagsensor import *
from sensors.calibration.camera_calibration import *


filename = "live_april"
start_time = time.time()
sensor = AprilTagSensor(cal_dir)
print(f"Sensor calibration and setup in {time.time()-start_time} seconds")
path = robot.take_picture(image_dir + "/testing/pose_test/")
measurements={}
start_time = time.time()
sensor.detect(path, measurements)
print(f"Sensor detections in {time.time()-start_time} seconds")
print(measurements)

AprilTag sensor setup COMPLETE
Sensor calibration and setup in 0.03280496597290039 seconds


error: OpenCV(4.1.1) /home/nvidia/host/build_opencv/nv_opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [ ]:
sys.path.append("../../deliveryrobot")

from navigation.slam.onlineslam import *

slam = OnlineSLAM(3)
slam.debug = False
start_time = time.time()
slam.process_measurements(measurements)
slam.process_movement(0,0)
slam.map_update()
print(f"SLAM sequence in {time.time()-start_time} seconds")

In [ ]:
from navigation.astar.astar import *
goal_state = measurements["4"]
obstacles = {}

scalar = 1
astar = Astar(
    5,
    np.pi/4,
    0.025*scalar,
    1000/scalar,
    1,
    1,
    0.015*scalar
)
astar.debug = False
astar.verbose = False
robot_state = np.array([0.,0.,0.])

start_time = time.time()
test = astar.astar_move(robot_state, obstacles, goal_state)
duration = time.time() - start_time
print(f"Results from Astar came in {duration} seconds and are:")
test.print_info()

In [ ]:
import kinematics.movementai as ai
steering = ai.SteeringOutput([0.05,0.0],0.0)
left, right = robot.robot_ai.get_drive_params(steering, 1)
print(left,right)
robot.robot.left(left)
robot.robot.right(right)
time.sleep(0.5)
robot.robot.stop()

In [ ]:
"""
robot.target_ai.slam_update([1,0,0])

start_time = time.time()
path_following = True

robot.update_path_ai([(0,[0.0,0.0]),(1,[0.5,0.0]),(2,[0.75, 0.25])])
while path_following:
    
    # TODO figure out what's going wrong here
    steering = robot.path_follow_ai(0.5)
    robot.robot_ai.estimate_update(steering, time.time() - start_time)
    start_time = time.time()
    
    if steering == None:
        path_following = False
        robot.left(0)
        robot.right(0)"""